In [27]:
import unittest
from unittest.mock import Mock, patch
import json
import datetime
import requests
from requests.exceptions import Timeout, ConnectionError
# https://realpython.com/lessons/python-mock-object-library-summary/

### What Is Mocking?

A mock object substitutes and imitates a real object within a testing environment. Replacing the actual request with a mock object would allow you to simulate external service outages and successful responses in a predictable way. Thus, it controls the code behavior during testing.

unnittest.mock provide a base class for mocking objects called Mock, which you will use to imitate real objects in your codebase

In [2]:
mock = Mock()
print("investigate mock object created:", mock)

investigate mock object created: <Mock id='140637683881632'>


A Mock simulate any object that it replaces. To achieve such flexibility, it creates attributes when access them. Aka, it could create new attributes that didn't exists (WARMING). 

Notice two key characteristics of this mocked version of dumps():

1. Unlike the real dumps(), this mocked method requires no arguments. In fact, it will accept any arguments that you pass to it.

2. The return value of dumps() is also a Mock. The capability of Mock to recursively define other mocks allows for you to use mocks in complex situations:

In [6]:
print(mock.some_attribute)

## let's try mock the entire json lib, so when we call json.dumps(), mock object will create the method .dumps to match the lib interface
json = Mock()
print(json.dumps())
# but when you make a typo of the real method, mock json will still be accessible
print(json.dumpyfy())

<Mock name='mock.some_attribute' id='140637683904808'>
<Mock name='mock.dumps()' id='140637683882920'>
<Mock name='mock.dumpyfy()' id='140637683882864'>


#### Attributes and Inspection
Mock instances store data on how you used them, so you can view the attributes to understand how your application used an object.

In [7]:
json = Mock()
json.loads(s='{"key": "value"}')

<Mock name='mock.loads()' id='140637681369328'>

In [8]:
# inspect mock usage 
print("num of times called loads", json.loads.call_count)
print("last loads call:", json.loads.call_args)
print("list of loads call:", json.loads.call_args_list)

num of times called loads 1
last loads call: call(s='{"key": "value"}')
list of loads call: [call(s='{"key": "value"}')]


#### Return Value and Side Effect
We can specify a function’s return value to control the code's behavior during tests.
When you want to make functions return different values when you call them more than once or define what happens when you call the mocked function (e.g. raise exception), can use .side_effect. 

In [18]:
# Save a couple of test days
tuesday = datetime.datetime(year=2019, month=1, day=1)
saturday = datetime.datetime(year=2019, month=1, day=5)

def is_weekday():
    today = datetime.datetime.today()
    return (0<= today.weekday() < 5)


def get_holidays():
    r = requests.get('http://localhost/api/holidays')
    if r.status_code == 200:
        return r.json()
    return None

In [16]:
print("AS of today is_weekday()", is_weekday())

datetime = Mock()
# Mock .today() to return Tuesday
datetime.datetime.today.return_value = tuesday
assert is_weekday()
# Mock .today() to return Saturday
datetime.datetime.today.return_value = saturday
assert not is_weekday()

AS of today is_weekday() True


#### Configuring Your Mock
Can configure a Mock to set up some of the object’s behaviors, e.g. .return_value, .side_effect, with .configure_mock()

In [22]:
mock = Mock()
mock.configure_mock(name = "Test mock", return_value=True)
mock()

True

In [25]:
holidays = {'12/25': 'Christmas', '7/4': 'Independence Day'}
response_mock = Mock(**{'json.return_value': holidays})
print(response_mock)
print(response_mock.json.return_value)

<Mock id='140637704625960'>
{'12/25': 'Christmas', '7/4': 'Independence Day'}


### patch()
patch() looks up an object in a given module and replaces that object with a Mock. And it can be used as a decorator or a context manager.

A good rule of thumb is to patch() the object where it is looked up.

In [29]:
class TestCalendar(unittest.TestCase):
    def test_get_holidays_timeout(self):
        with patch('my_calendar.requests') as mock_requests:
            mock_requests.get.side_effect = Timeout
            with self.assertRaises(Timeout):
                get_holidays()
                mock_requests.get.assert_called_once()

#### with patch.object(), you can mock one method instead of the entire object.
object() takes the same configuration params that patch() does, as the first param is the path to the target object. The second parameter is the attribute of the target object that plan to mock.

In [28]:
class TestCalendar(unittest.TestCase):
    @patch.object(requests, 'get', side_effect=requests.exceptions.Timeout)
    def test_get_holidays_timeout(self, mock_requests):
            with self.assertRaises(requests.exceptions.Timeout):
                get_holidays()

### Use Specification to avoid common problem
As mock creates new attributes and methods when you access, even misspelled or not existing attributes, you can still able to test, though not meaningful. 

To prevent, you can pass an object specification to the `spec` parameter, which accepts a list of names or another object and defines the mock’s interface. If you attempt to access an attribute that does not belong to the specification, Mock will raise an AttributeError

In [30]:
calendar = Mock(spec=['is_weekday', 'get_holidays'])

In [32]:
print(calendar.is_weekday())
print(calendar.not_available_func())

<Mock name='mock.is_weekday()' id='140637715496296'>


AttributeError: Mock object has no attribute 'not_available_func'